In [5]:
#In this workbook, sentiment of reviews will be measured using the stars and then the text of the review. 
#The sentiment will be mapped with dates and be visualized through three years.
#Cluster analysis - topic modeling - What are people talking about?
# What are my questions?

#Q. In last three years, what are the most popular words that people have used to describe maggie?
#Q. How has the sentiment evolved over a period of three years?
#Q. 

#Possible topics - taste, texture, packaging, time of arrival

In [6]:
# -*- coding: utf-8 -*-

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

In [28]:
#read the cleaned csv

#maggie = pd.read_csv("maggie.csv")
maggie = pd.read_csv("earphones.csv")
maggie["comment"] = maggie["comment"].astype(str)
#maggie["date"] = pd.to_datetime(maggie["date"], format='%Y-%m-%d')
#maggie["date"] = maggie["date"].dt.date

In [29]:
maggie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    4990 non-null   int64 
 1   comment  4990 non-null   object
 2   date     4990 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.1+ KB


In [30]:
maggie["date"].max()

'2020-08-16'

In [31]:
maggie["date"].min()

'2018-08-22'

In [32]:
#Clean text column -- remove stopwords, punctuation
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [33]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
exclusionList = ['maggie','product','would','kar','ki','noodles', 'sn', 'maggi', "pack", "noodle", "et", "test", "ed"]
exclusions = '|'.join(exclusionList)

# cleaning master function
def clean_text(text, bigrams = False):
    text = text.lower() # lower case
    text = re.sub('['+my_punctuation + ']+', ' ', text) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    text = re.sub(exclusions, '', text) #remove common words like maggie and noodles
    text_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords

    #text_token_list = [word_rooter(word) if '#' not in word else word
    #                    for word in text_token_list] # apply word rooter
    #if bigrams:
    #    text_token_list = text_token_list+[text_token_list[i]+'_'+text_token_list[i+1]
    #                                        for i in range(len(text_token_list)-1)]
    text = ' '.join(text_token_list)
    return text

In [34]:
maggie['comment'] = maggie.comment.apply(clean_text)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df = 25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(maggie['comment']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

In [36]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 4

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [37]:
model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [39]:
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights
0,good,1859.7,jbl,463.2,worng,562.5,ear,580.9
1,sound,1180.9,bluooth,267.3,months,370.4,sound,509.9
2,quality,1099.7,earphones,218.3,one,359.1,quality,507.3
3,battery,678.2,worth,180.2,month,311.2,ears,265.0
4,noise,345.8,quality,137.0,worst,258.5,good,255.2
5,bass,308.3,’s,135.2,days,250.1,bass,234.0
6,cancellation,272.6,connect,122.6,jbl,243.6,best,219.7
7,life,228.0,nice,121.6,buy,236.2,earphones,194.3
8,poor,209.1,one,120.4,bad,233.4,fit,191.2
9,backup,205.2,connectivity,116.2,quality,229.0,use,176.3


In [ ]:
#Classify the sentiments --Positive and negative
#Simple frequency analysis - Most popular words in reviews


In [ ]:
#Stars are in the range of 1-5. Anything between below 3 will be considered a negative sentiment, 3 as neutral and 
#above 3 as positive.

#Count #of 1's, 2's and 3's and 4's and 5's in ratings and create a separate cluster

#Step 1: Create a new colummn labelling the sentiments as positive, negative and neutral
